In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hwb

     |████████████████████████████████| 1.4 MB 7.8 MB/s 


In [3]:
import cv2
import random
import json
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import torch, torchvision
import warnings
from skimage import io
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torchvision
from torch.nn.utils.rnn import pad_sequence

warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import shutil
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms


In [4]:
import torch
from torch import nn

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
!unzip -q '/content/drive/MyDrive/НТИ ИИ /data_team/data_final.zip'

## Build datasets

In [6]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import re

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, transforms):
        self.root_dir = root_dir
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def _load_file(self, path):
      image = cv2.imread(path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      return image
      
    def __getitem__(self, idx):
        el = self.df.iloc[idx]
        file_name = el['image']

        image = self._load_file(os.path.join(self.root_dir, file_name))
        image = self.transforms(image=image)['image']
        
        return image, el['label']

In [7]:
# from augmixations import HandWrittenBlot
from hwb import HandWrittenBlot
import albumentations as A


class AlbuHandWrittenBlot(A.DualTransform):
    def __init__(self, hwb, always_apply=False, p=0.5):
        super(AlbuHandWrittenBlot, self).__init__(always_apply, p)
        self.hwb = hwb

    def apply(self, image, **params):
        return self.hwb(image)


class AlbuPadding(A.DualTransform):
    def __init__(self, always_apply=False, p=0.5):
        super(AlbuPadding, self).__init__(always_apply, p)

    def apply(self, image, **params):
        zeros = np.zeros((128, 384, 3))
        image = np.concatenate([zeros, image, zeros], axis=0)
        return image.astype(np.uint8)

In [8]:
rectangle_info = {
    'x': (None, None),
    'y': (None, None),
    'h': (60, 100),  
    'w': (50, 80),
}

blot_params = {
    'incline': (-10, 10),
    'intensivity': (0.5, 0.9),
    'transparency': (0.05, 0.4),
    'count': (1, 3),
}

blots = HandWrittenBlot(rectangle_info, blot_params)

In [43]:
from albumentations.pytorch.transforms import ToTensor

data_transforms = {
    'train': A.Compose([
              A.Resize(256, 256),
              AlbuHandWrittenBlot(blots, p=0.3),
              A.Rotate(limit=[-7, 7]),
              A.OneOf([
                A.ToGray(always_apply=True),
                A.CLAHE(always_apply=True, clip_limit=15),
              ], 0.3),
              ToTensor()
          ]),
    'val': A.Compose([
              A.Resize(256, 256),
              ToTensor()
          ]),
}

In [12]:
hack_data = pd.read_csv('data/train_recognition/labels.csv')
del hack_data['base_image']
hack_data.columns = ['image', 'label']
hack_data['image'] = hack_data['image'].apply(lambda x: os.path.join('data/train_recognition/images', x))
hack_data = hack_data[hack_data['label'].apply(lambda x: len(x) >= 4)]
hack_data['label'] = hack_data['label'].apply(lambda x: re.search('[а-яА-Я]', x) is not None).astype(int)

In [13]:
sum(hack_data['label'] == 0), sum(hack_data['label'] == 1)

(9440, 98944)

In [14]:
hack_data = pd.concat([hack_data[hack_data['label'] == 1][:10000], hack_data[hack_data['label'] == 0][:10000]])
len(hack_data)

19440

In [15]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(hack_data, test_size=0.1, random_state=42, shuffle=True)

In [16]:
len(df_train)

17496

In [44]:
train_dataset = IAMDataset(root_dir='./',
                           df=df_train,
                           transforms=data_transforms['train'])

val_dataset = IAMDataset(root_dir='./',
                         df=df_val,
                         transforms=data_transforms['val'])

In [45]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))

Number of training examples: 17496
Number of validation examples: 1944


In [46]:
from torch.utils.data import DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16,)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16,)

## Train loops

In [23]:
def train_epoch(model, batch_gen, criterion, optimizer, is_train = True) :
  epoch_loss = 0.0
  count = 0
  accuracy = 0
  model.train(is_train)
  
  for input, labels in tqdm(batch_gen) :
    input = input.to(device)
    labels = labels.to(device)
    
    with torch.set_grad_enabled(is_train) :
      pred = model(input)
      loss = criterion(pred, labels)
      accuracy += torch.sum(torch.max(pred, 1)[1] == labels)
      
      if is_train :
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
      count += input.size(0)
      epoch_loss += loss.item()

  return epoch_loss / count, accuracy / count


In [21]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs) :
  loader = {'train' : train_loader, 'test' : test_loader}
  loss_history = {'train' : [], 'test' : []}
  acc_history = {'train' : [], 'test' : []}

  for epoch in range(num_epochs) :
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    for phase in ['train', 'test'] :
      epoch_loss, epoch_acc = train_epoch(model, loader[phase], criterion, optimizer, phase == 'train')
      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
      loss_history[phase].append(epoch_loss)
      acc_history[phase].append(epoch_acc)
    
    print()
  
  return loss_history, acc_history

## Train model

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

In [48]:
model.fc = nn.Sequential(
    nn.Linear(2048, 256),
    nn.ReLU(),
    nn.Linear(256, 2),
)

In [49]:
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

In [50]:
loss_train, acc_train = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

Epoch 0/2
----------


100%|██████████| 2187/2187 [03:59<00:00,  9.13it/s]


train Loss: 0.0216 Acc: 0.9351


100%|██████████| 243/243 [00:10<00:00, 23.53it/s]


test Loss: 0.0118 Acc: 0.9568

Epoch 1/2
----------


...

test Loss: 0.0058 Acc: 0.9789

Epoch 2/2
----------


100%|██████████| 2187/2187 [03:56<00:00,  9.24it/s]


train Loss: 0.0089 Acc: 0.9748


100%|██████████| 243/243 [00:10<00:00, 23.68it/s]

test Loss: 0.0046 Acc: 0.9866



In [51]:
torch.save(model, "/content/drive/MyDrive/НТИ ИИ /team/sergey_models/language_classifier_256")